In [1]:
import pandas as pd
import pickle
import math
import copy
import numpy as np

In [2]:
clusters = 10

In [3]:
years = [2030]

# NUTS-Regionen

In [6]:
for year in years:
    
    # Read profiles
    data_profiles = pd.read_pickle("data/profiles_daily_clusters" + str(clusters) + ".pickle")
    data_profiles_agg = data_profiles[list(data_profiles.keys())[0]][year].fillna(0)
    for key in list(data_profiles.keys())[1:]:
        data_profiles_agg = data_profiles_agg + data_profiles[key][year].fillna(0)
    data_profiles_agg = data_profiles_agg.apply(lambda x: round(x, 2))

    # Read flex profiles (positive)
    data_flex_pos = pd.read_pickle("data/pos_flex_daily_clusters" + str(clusters) + ".pickle")
    data_flex_pos_agg = data_flex_pos[list(data_flex_pos.keys())[0]][year].fillna(0)
    for key in list(data_flex_pos.keys())[1:]:
        data_flex_pos_agg = data_flex_pos_agg + data_flex_pos[key][year].fillna(0)
    data_flex_pos_agg = data_flex_pos_agg.fillna(0).apply(lambda x: round(x, 2))

    # Read flex profiles (negative)
    data_flex_neg = pd.read_pickle("data/neg_flex_daily_clusters" + str(clusters) + ".pickle")
    data_flex_neg_agg = data_flex_neg[list(data_flex_neg.keys())[0]][year].fillna(0)
    for key in list(data_flex_neg.keys())[1:]:
        data_flex_neg_agg = data_flex_neg_agg + data_flex_neg[key][year].fillna(0)
    data_flex_neg_agg = data_flex_neg_agg.fillna(0).apply(lambda x: round(x, 2))
    
    # Read driving profiles
    data_driving = pd.read_pickle("data/driving_daily_clusters" + str(clusters) + ".pickle")
    data_driving_agg = data_driving[list(data_driving.keys())[0]][year].fillna(0)
    for key in list(data_driving.keys())[1:]:
        data_driving_agg = data_driving_agg + data_driving[key][year].fillna(0)
    data_driving_agg = data_driving_agg.fillna(0).apply(lambda x: round(x, 2))
    
    data_driving_agg_ref = copy.deepcopy(data_driving_agg)
    for c in range(clusters):
        for i in range(8760):
            i_start = 24*math.floor(i/24)
            i_end = i_start+24
            if sum(data_driving_agg_ref.iloc[i_start:i_end,c]) > 0:
                fac = sum(data_profiles_agg.iloc[i_start:i_end,c]) / sum(data_driving_agg_ref.iloc[i_start:i_end,c])
            else:
                fac = 1   
            data_driving_agg.iloc[i,c] = data_driving_agg_ref.iloc[i,c] * fac
     
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter("Output/profiles_nuts_clusters" + str(clusters) + "_year" + str(year) +".xlsx", engine='xlsxwriter')

    # Write each dataframe to a different worksheet.
    data_profiles_agg.to_excel(writer, sheet_name='profiles')
    data_flex_neg_agg.to_excel(writer, sheet_name='flex_neg')
    data_flex_pos_agg.to_excel(writer, sheet_name='flex_pos')
    data_driving_agg.to_excel(writer, sheet_name='driving')

    # Close the Pandas Excel writer and output the Excel file.
    writer.close()

# Siedlungstypen

In [46]:
# Read profiles
data_profiles = pd.read_pickle("data/profiles_daily_Siedlungstyp_clusters" + str(clusters) + ".pickle")
data_profiles = pd.concat([pd.concat({str(i): pd.concat(data_profiles[i], axis=1)}, axis=1) for i in data_profiles],axis=1)
data_profiles = data_profiles.reorder_levels([1, 0, 2], axis=1)
data_profiles = data_profiles.sort_index(axis=1)
data_profiles = data_profiles.apply(lambda x: round(x, 4))

# Read flex profiles (positive)
data_flex_pos = pd.read_pickle("data/pos_flex_daily_Siedlungstyp_clusters" + str(clusters) + ".pickle")
data_flex_pos = pd.concat([pd.concat({str(i): pd.concat(data_flex_pos[i], axis=1)}, axis=1) for i in data_flex_pos],axis=1)
data_flex_pos = data_flex_pos.reorder_levels([1, 0, 2], axis=1)
data_flex_pos = data_flex_pos.sort_index(axis=1)
data_flex_pos = data_flex_pos.apply(lambda x: round(x, 4))

# Read flex profiles (negative)
data_flex_neg = pd.read_pickle("data/neg_flex_daily_Siedlungstyp_clusters" + str(clusters) + ".pickle")
data_flex_neg = pd.concat([pd.concat({str(i): pd.concat(data_flex_neg[i], axis=1)}, axis=1) for i in data_flex_neg],axis=1)
data_flex_neg = data_flex_neg.reorder_levels([1, 0, 2], axis=1)
data_flex_neg = data_flex_neg.sort_index(axis=1)
data_flex_neg = data_flex_neg.apply(lambda x: round(x, 4))

# Read driving profiles
data_driving = pd.read_pickle("data/driving_daily_Siedlungstyp_clusters" + str(clusters) + ".pickle")
data_driving = pd.concat([pd.concat({str(i): pd.concat(data_driving[i], axis=1)}, axis=1) for i in data_driving],axis=1)
data_driving = data_driving.reorder_levels([1, 0, 2], axis=1)
data_driving = data_driving.sort_index(axis=1)
data_driving = data_driving.apply(lambda x: round(x, 4))

data_driving_ref = copy.deepcopy(data_driving)
for c, k in enumerate(data_driving.keys()):
    for i in range(8760):
        i_start = 24*math.floor(i/24)
        i_end = i_start+24
        if sum(data_driving_ref.iloc[i_start:i_end,c]) > 0:
            fac = sum(data_profiles.iloc[i_start:i_end,c]) / sum(data_driving_ref.iloc[i_start:i_end,c])
        else:
            fac = 1
        data_driving.iloc[i,c] = data_driving_ref.iloc[i,c] * fac

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter("Output/profiles_siedlungstyp_clusters" + str(clusters) + ".xlsx", engine='xlsxwriter')

# Write each dataframe to a different worksheet.
data_profiles.to_excel(writer, sheet_name='profiles')
data_flex_neg.to_excel(writer, sheet_name='flex_neg')
data_flex_pos.to_excel(writer, sheet_name='flex_pos')
data_driving.to_excel(writer, sheet_name='driving')

# Close the Pandas Excel writer and output the Excel file.
writer.close()